In [1]:
import pandas as pd


In [2]:
df = pd.read_csv('EmaSp.csv')

UnicodeDecodeError: 'utf-8' codec can't decode bytes in position 617-618: invalid continuation byte

In [3]:
import chardet

In [4]:
with open('EmaSp.csv', 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))
result

{'encoding': 'Windows-1252', 'confidence': 0.7270329257462941, 'language': ''}

In [5]:
df = pd.read_csv('EmaSp.csv', encoding = 'Windows-1252' )

In [6]:
df.head()

,Category,Message,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [7]:
df.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],  axis = 'columns' , inplace = True)

In [8]:
df.groupby('Category').describe()

Message                                                            \
           count unique                                                top   
Category                                                                     
ham         4823   4514                             Sorry, I'll call later   
spam         747    653  Please call our customer service representativ...   

               
         freq  
Category       
ham        30  
spam        4

In [9]:
df['spam'] = df['Category'].apply(lambda x: 1  if x=='spam' else 0)
df.head()

,Category,Message,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [10]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df.Message,df.spam, test_size = 0.25)

In [11]:
x_train.values

array(['Hows that watch resizing',
       'SMS SERVICES. for your inclusive text credits, pls goto www.comuk.net login= 3qxj9 unsubscribe with STOP, no extra charge. help 08702840625.COMUK. 220-CM2 9AE',
       'For taking part in our mobile survey yesterday! You can now have 500 texts 2 use however you wish. 2 get txts just send TXT to 80160 T&C www.txt43.com 1.50p',
       ...,
       'Had the money issue weigh me down but thanks to you, I can breathe easier now. I.ll make sure you dont regret it. Thanks.',
       "The table's occupied, I'm waiting by the tree",
       "K I'll be there before 4."], dtype=object)

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
v = CountVectorizer()
x_train_count = v.fit_transform(x_train.values)
x_train_count.toarray()[:3]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [13]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(x_train_count,y_train)

MultinomialNB()

In [14]:
mails = [
    'Babab suwe je elere okunrin 30%',
    'upto 25% discount on parking, exlusive offer just for you. Dont misst this reward!'
]
emails_count = v.transform(mails)
model.predict(emails_count)

array([0, 1], dtype=int64)

In [15]:
x_test_count =v.transform(x_test)
model.score(x_test_count, y_test)

0.9856424982053122

In [18]:
from sklearn.pipeline import Pipeline
clf = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('nb', MultinomialNB())
])

In [19]:
clf.fit(x_train, y_train)

Pipeline(steps=[('vectorizer', CountVectorizer()), ('nb', MultinomialNB())])

In [20]:
 clf.score(x_train, y_train)

0.9930572180991142

In [22]:
clf.predict(x_test)

array([0, 0, 0, ..., 0, 0, 1], dtype=int64)